## search engine with tools and agents

In [7]:
from langchain_community.tools import ArxivQueryRun , WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper , ArxivAPIWrapper


In [8]:
wiki_wrapper= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki.name


'wikipedia'

In [9]:
arxiv_wrapper= ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [10]:
tools = [arxiv,wiki]

In [11]:
# custom
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader= WebBaseLoader("https://python.langchain.com/docs/introduction/")
docs=loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model="gemma:2b"))
retriever=vectordb.as_retriever()
retriever

C:\Users\hp\AppData\Local\Temp\ipykernel_16440\2459704521.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model="gemma:2b"))


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018FC5BF6600>, search_kwargs={})

In [13]:
from langchain.tools.retriever import create_retriever_tool
ret_tool=create_retriever_tool(retriever,"langchain-search","search any info about langchain")
ret_tool.name

'langchain-search'

In [16]:
tools=[wiki,arxiv,ret_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\langchain\\lenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langchain-search', description='search any info about langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000018FAB23F2E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018FC5BF6600>, search_kwargs={}), docume

## run these tools with agents and llms

In [26]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
import os
api_key=os.getenv("GROQ_API_KEY")
llm = ChatGroq(api_key=api_key,model="llama-3.3-70b-versatile")


In [27]:
#prompt
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [28]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
# to execute tools , prompt , llm all together as chain we need agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [30]:
#executer
from langchain.agents import AgentExecutor
agentexec=AgentExecutor(agent=agent,tools=tools,verbose=True)
agentexec

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [42]:
ans= agentexec.invoke({"input":"who is virat kohli"})
ans



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Virat Kohli'}`


Page: Virat Kohli
Summary: Virat Kohli (Hindi pronunciation: [ʋɪˈɾaːʈᵊ ˈkoːɦᵊliː] , born 5 November 1988) is an Indian international cricketer and the former captain of the Indian national cricket team. He is a right-handed batsman and an occasional Virat Kohli is an Indian international cricketer and former captain of the Indian national cricket team. He is a right-handed batsman and occasional right-arm medium-pace bowler. Kohli is widely regarded as one of the greatest batsmen of all time, known for his exceptional technique, consistency, and ability to score runs in all formats of the game. He has won numerous awards and accolades, including the ICC Cricketer of the Year award, and has been named the Indian Sports Honours' Sportsman of the Year. Kohli is also known for his aggressive and passionate on-field personality, as well as his philanthropic efforts off the field, particularly in

{'input': 'who is virat kohli',
 'output': "Virat Kohli is an Indian international cricketer and former captain of the Indian national cricket team. He is a right-handed batsman and occasional right-arm medium-pace bowler. Kohli is widely regarded as one of the greatest batsmen of all time, known for his exceptional technique, consistency, and ability to score runs in all formats of the game. He has won numerous awards and accolades, including the ICC Cricketer of the Year award, and has been named the Indian Sports Honours' Sportsman of the Year. Kohli is also known for his aggressive and passionate on-field personality, as well as his philanthropic efforts off the field, particularly in the area of children's education and healthcare."}